In [1]:
import os

In [2]:
%pwd

'd:\\AI\\Kidney-Disease-Classification-MLflow-DVC\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\AI\\Kidney-Disease-Classification-MLflow-DVC'

In [38]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareModelConfiq:
    root_dir: Path
    base_model_path: Path
    base_model_updated_path: Path
    parms_image_size: list
    parms_learning_rate: float
    parms_include_top: bool
    parms_weights: str
    parms_classes: int

In [39]:
from cnnClassifier.constants import *
from src.cnnClassifier.utils.comman import read_yaml, create_path

In [40]:
class ConfigurationManager:
    def __init__(
            self,
            config_path=CONFIG_FILE_PATH,
            parms_path=PARMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.parms = read_yaml(parms_path)

        create_path([self.config.artifacts_root])

    def get_prepare_base_model_config(self):
        config = self.config.  
        create_path([config.root_dir])
        prepare_base_model_config = PrepareModelConfiq(
            root_dir = Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            base_model_updated_path=Path(config.base_model_updated_path),
            parms_image_size=self.parms.IMAGE_SIZE,
            parms_learning_rate=self.parms.LEARNING_RATE,
            parms_include_top=self.parms.INCLUDE_TOP,
            parms_weights=self.parms.WEIGHTS,
            parms_classes=self.parms.CLASSES
        )       
        return prepare_base_model_config

In [41]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [44]:
class PrepareBaseModel:
    def __init__(self,config = PrepareModelConfiq):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.parms_image_size,
            include_top=self.config.parms_include_top,
            weights= self.config.parms_weights
        )
        self.save_model(path=self.config.base_model_path,model=self.model)

    #@staticmethod
    def prepare_full_model(self,model,classes,freeze_all,freeze_till,learning_rate):
        if freeze_all:
            for layers in model.layers:
                model.trained=False
        elif (freeze_all is not None) and (freeze_till>0):
            for layer in model.layers[:-freeze_till]:
                model.trained=False
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation='softmax'
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
            )
        
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']
            )
        
        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self.prepare_full_model(
            model=self.model,
            classes=self.config.parms_classes,  # corrected from params_classes to parms_classes
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.parms_learning_rate
        )
        self.save_model(path=self.config.base_model_updated_path, model=self.full_model)


    
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [45]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

config\config.yaml
[2024-08-17 03:50:37,827: INFO: comman: yaml file:config\config.yaml load sucessfully]
params.yaml
[2024-08-17 03:50:37,832: INFO: comman: yaml file:params.yaml load sucessfully]
[2024-08-17 03:50:37,835: INFO: comman: Directorys:artifacts sucessfully]
[2024-08-17 03:50:37,837: INFO: comman: Directorys:artifacts/prepare_base_model sucessfully]
[2024-08-17 03:50:40,490: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D